# Day 15: Science for Hungry People

[*Advent of Code 2015 day 15*](https://adventofcode.com/2015/day/15) and [*solution megathread*](https://www.reddit.com/3wwj84)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2015/15/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2015%2F15%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

## Comments

I hear there is no good solution for this except just repeated attempts (more or less smart...). So, if I'm to succeed at this, I expect I _must_ use numpy... so let's get to it?

In [5]:
testdata = """Butterscotch: capacity -1, durability -2, flavor 6, texture 3, calories 8
Cinnamon: capacity 2, durability 3, flavor -2, texture -1, calories 3"""

inputdata = downloaded['input']

1:80: E501 line too long (87 > 79 characters)


In [6]:
print(f'{inputdata[:200]}...')

Sugar: capacity 3, durability 0, flavor 0, texture -3, calories 2
Sprinkles: capacity -3, durability 3, flavor 0, texture 0, calories 9
Candy: capacity -1, durability 0, flavor 4, texture 0, calories ...


In [7]:
import re
import numpy as np
import pandas as pd


def parse_data(data: str):
    output = pd.DataFrame()
    pattern = re.compile(
        r'^(?P<ingredient>\w+): capacity (?P<capacity>-?\d+), ' +
        r'durability (?P<durability>-?\d+), flavor (?P<flavor>-?\d+), ' +
        r'texture (?P<texture>-?\d+), calories (?P<calories>-?\d+)$')
    for line in data.splitlines():
        ingredient = pattern.match(line).groupdict()
        ingredient_name = ingredient["ingredient"]
        del ingredient["ingredient"]
        for key in ingredient.keys():
            ingredient[key] = int(ingredient[key])
        output = pd.concat(
            [output,
             pd.Series(ingredient,
                       name=ingredient_name).to_frame().T],
            )
    return output

In [8]:
def values_by_ingredient(
        composition: pd.Series,
        nutrition: pd.DataFrame) -> pd.DataFrame:
    return nutrition.mul(composition, axis='index')


def nutritional_value(
        composition: pd.Series,
        nutrition: pd.DataFrame) -> int:
    values = values_by_ingredient(
        composition,
        nutrition)
    values_by_property = values.loc[
        :, values.columns != 'calories'].sum()
    if any(values_by_property <= 0):
        return 0
    else:
        return values_by_property.prod()

In [9]:
from IPython.display import display


nutrition = parse_data(testdata)
display(nutrition)
composition = pd.Series([44, 56], name="amount", index=nutrition.index)
nutritional_value(composition, nutrition)

,capacity,durability,flavor,texture,calories
Butterscotch,-1,-2,6,3,8
Cinnamon,2,3,-2,-1,3


62842880

In [10]:
from math import floor


# This solution unfortunately did not terminate,
# since I don't check whether any ingredient went
# below zero...
def improve_composition(
        composition: pd.Series,
        nutrition: pd.DataFrame,
        debug: bool = False) -> (pd.Series, int):
    best_value = nutritional_value(composition, nutrition)
    best_composition = composition
    number_of_ingredients = len(composition)
    for ingredient in composition.index:
        test_composition = composition.copy()
        test_composition[ingredient] += number_of_ingredients - 1
        test_composition[test_composition.index != ingredient] -= 1
        test_value = nutritional_value(test_composition, nutrition)
        if test_value > best_value:
            best_value = test_value
            best_composition = test_composition
    if debug:
        print(f'improved: {best_value} - {best_composition}')
    return best_composition, best_value

In [11]:
# This starting position is not guaranteed to be
# even positive, so difficult to improve
def starting_composition(
        nutrition: pd.DataFrame) -> pd.Series:
    number_of_ingredients = len(nutrition.index)
    fraction = floor(100 / number_of_ingredients)
    startlist = [fraction] * number_of_ingredients
    startlist[0] += 100 % number_of_ingredients
    return pd.Series(startlist, name="amount", index=nutrition.index)

In [12]:
from random import randint, random


def randomly_change_composition(
        composition: pd.Series,
        nutrition: pd.DataFrame) -> (pd.Series, int):
    take_from = composition.loc[
        composition > 0].sample()
    give_to = composition.drop(
        index=take_from.index).sample()
    new_composition = composition.copy()
    amount = floor(0.2 * random() * take_from)
    new_composition[take_from.index] -= amount
    new_composition[give_to.index] += amount
    return (
        new_composition,
        nutritional_value(new_composition, nutrition))


def randomly_improve_composition(
        composition: pd.Series,
        nutrition: pd.DataFrame,
        attempts: int = 400,
        debug: bool = False) -> (pd.Series, int):
    best_value = nutritional_value(composition, nutrition)
    best_composition = composition
    failed_attempts = 0
    while failed_attempts < attempts:
        test_composition, test_value = \
            randomly_change_composition(
                best_composition,
                nutrition)
        if test_value <= best_value:
            failed_attempts += 1
        else:
            best_value = test_value
            best_composition = test_composition
            failed_attempts = 0
    if debug:
        print(f'improved: {best_value} - {best_composition}')
    return best_composition, best_value


def random_starting_composition(
        nutrition: pd.DataFrame) -> pd.Series:
    fractions = [
        floor(random() * 100)
        for _ in range(len(nutrition.index))]
    fractions[0] += 100 - sum(fractions)
    composition = pd.Series(
        fractions,
        name="amount",
        index=nutrition.index)
    return composition, nutritional_value(
        composition, nutrition)


def good_starting_composition(
        nutrition: pd.DataFrame) -> pd.Series:
    while True:
        composition, value = \
            random_starting_composition(
                nutrition)
        if value > 0:
            return composition


def my_part1_solution(
        data: str,
        debug: bool = False) -> (pd.Series, int):
    nutrition = parse_data(data)
    composition = good_starting_composition(nutrition)
    return randomly_improve_composition(
        composition,
        nutrition)

In [13]:
my_part1_solution(inputdata, True)

(Sugar        21
 Sprinkles     5
 Candy        31
 Chocolate    43
 Name: amount, dtype: int64,
 222870)

In [14]:
HTML(downloaded['part1_footer'])

## Part Two

In [15]:
HTML(downloaded['part2'])

In [16]:
# HTML(downloaded['part2_footer'])

In [17]:
# assert(my_part2_solution(testdata) == ...)

In [18]:
# my_part2_solution(inputdata)